# Crop and Weed Detection in real-time video using deep learning

## below steps are for data visualization and preprocessing

!wget https://github.com/KevalPadsala/Crop_and_Weed_detection/raw/main/crop_and_weed_detection_dataset.zip

In [ ]:
!unzip crop_and_weed_detection_dataset.zip

## Becoming one with the data

In [ ]:
# Get the file names from the dataset
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle

def show_random_image(path="/content/agri_data/data"):
  file_names = []
  for file in os.listdir(path):
    if ".jpeg" in file:
      file_names.append(file)

  # Generating a random images

  random_image_name = random.choice(file_names)
  # print(random_image_name)
  img = mpimg.imread(path + "/" + random_image_name)
  plt.imshow(img)
  raw_random_image = random_image_name[:-5]
  # print(raw_random_image)

  random_image_path = path + "/" + raw_random_image + ".txt"

  with open(random_image_path) as f:
    content_list = f.readlines()

  # remove new line characters
  content_list = [x.strip() for x in content_list]

  if content_list[0][0] == 0:
    category = "crop"
  else:
    category = "weed"

  coordinates =  content_list[0].split()
  coordinates = coordinates[1:]
  x_cen = float(coordinates[0])
  y_cen = float(coordinates[1])
  w = float(coordinates[2])
  h = float(coordinates[3])

  x_min = int((x_cen - w/2)*512)
  y_min = int((y_cen - h/2)*512)
  x_max = int((x_cen + w/2)*512)
  y_max = int((x_cen + h/2)*512)

  plt.plot([x_min, x_max, x_max, x_min, x_min],[y_min, y_min, y_max, y_max, y_min])
  print(coordinates)
  plt.title(category)
  return img

In [ ]:
img = show_random_image()

In [ ]:
img.shape

In [ ]:
path="/content/agri_data/data"
from pathlib import Path
import shutil

img_names = []
for file in os.listdir(path):
  if ".jpeg" in file:
    img_names.append(file)
# print(img_names[:5])

file_names = []
for file in os.listdir(path):
  if ".txt" in file:
    file_names.append(file)
# print(file_names[:5])
for file in file_names:
  f = open(path + "/" + file, "r")
  content = f.read()
  if content[0] == "0":
    source = path + "/" + file[:-4] + ".jpeg"
    destination = "/content/preprocessing/crop_img/"
    # Path(source).rename(destination)
    shutil.move(source, destination)
  elif content[0] == "1":
    src = path + "/" + file[:-4] + ".jpeg"
    dst = "/content/preprocessing/weed_img/"
    shutil.move(src, dst)


## run this code for checking

In [2]:
!wget https://github.com/KevalPadsala/Crop_and_Weed_detection/raw/main/preprocessing.zip

--2023-09-10 15:51:10--  https://github.com/KevalPadsala/Crop_and_Weed_detection/raw/main/preprocessing.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/KevalPadsala/Crop_and_Weed_detection/main/preprocessing.zip [following]
--2023-09-10 15:51:10--  https://raw.githubusercontent.com/KevalPadsala/Crop_and_Weed_detection/main/preprocessing.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82705408 (79M) [application/zip]
Saving to: ‘preprocessing.zip’

preprocessing.zip   100%[===================>]  78.87M   308MB/s    in 0.3s    

2023-09-10 15:51:13 (308 MB/s) - ‘preprocessing.zip’ save

In [3]:
!unzip preprocessing.zip

Archive:  preprocessing.zip
   creating: preprocessing/test/
   creating: preprocessing/test/crop_img/
  inflating: preprocessing/test/crop_img/agri_0_1026.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_110.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1129.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_114.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1174.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1214.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1260.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_131.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1351.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_14.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1463.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1477.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1499.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_164.jpeg  
  inflating: preprocessing/test/crop_img/agri_0_1810.jpeg  
  i

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import sklearn.metrics as metrics
import seaborn as sns
import matplotlib.pyplot as plt

# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "preprocessing/train/"
test_dir = "preprocessing/test/"

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

# Create a CNN model
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10,
                         kernel_size=3,
                         activation="relu",
                         input_shape=(224, 224, 3)), #  input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compile the model
model_1.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=10,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Found 1060 images belonging to 2 classes.
Found 240 images belonging to 2 classes.
Epoch 1/10
34/34 [==============================] - 17s 159ms/step - loss: 0.4916 - accuracy: 0.7821 - val_loss: 0.3922 - val_accuracy: 0.8792
Epoch 2/10
34/34 [==============================] - 4s 123ms/step - loss: 0.3693 - accuracy: 0.8642 - val_loss: 0.4011 - val_accuracy: 0.8417
Epoch 3/10
34/34 [==============================] - 4s 114ms/step - loss: 0.3117 - accuracy: 0.8981 - val_loss: 0.3109 - val_accuracy: 0.8958
Epoch 4/10
34/34 [==============================] - 5s 140ms/step - loss: 0.2971 - accuracy: 0.8991 - val_loss: 0.2947 - val_accuracy: 0.9042
Epoch 5/10
34/34 [==============================] - 4s 114ms/step - loss: 0.2531 - accuracy: 0.9132 - val_loss: 0.2660 - val_accuracy: 0.9083
Epoch 6/10
34/34 [==============================] - 5s 148ms/step - loss: 0.2201 - accuracy: 0.9274 - val_loss: 0.2367 - val_accuracy: 0.9042
Epoch 7/10
34/34 [==============================] - 4s 117ms/ste

In [5]:
import numpy as np
pred_probs = model_1.predict(valid_data)
pred_probs = tf.squeeze(pred_probs)
np.round(pred_probs)

8/8 [==============================] - 1s 113ms/step


array([1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1.,
       0., 0., 1., 0., 1.

In [6]:
model_1.save('crop_weed_detection_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
from google.colab import files

model_path = 'crop_weed_detection_model.h5'

files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_dir = "preprocessing/test/"

test_data = test_datagen.flow_from_directory(test_dir,
                                             batch_size=32,
                                             target_size=(224, 224),
                                             class_mode="binary",
                                             seed=42)



Found 240 images belonging to 2 classes.


In [9]:
test_loss, test_accuracy = model_1.evaluate(test_data, steps=len(test_data))
print(f"Test Loss: {test_loss:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")


8/8 [==============================] - 1s 84ms/step - loss: 0.1961 - accuracy: 0.9208
Test Loss: 0.20
Test Accuracy: 0.92
